In [1]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [4]:
import pandas as pd
import numpy as np

In [5]:
wv.most_similar('king')

[('kings', 0.7138045430183411),
 ('queen', 0.6510956883430481),
 ('monarch', 0.6413194537162781),
 ('crown_prince', 0.6204220056533813),
 ('prince', 0.6159993410110474),
 ('sultan', 0.5864824056625366),
 ('ruler', 0.5797567367553711),
 ('princes', 0.5646552443504333),
 ('Prince_Paras', 0.5432944297790527),
 ('throne', 0.5422105193138123)]

In [16]:
df=pd.read_csv("https://raw.githubusercontent.com/aws-samples/aws-machine-learning-university-accelerated-nlp/master/data/examples/AMAZON-REVIEW-DATA-CLASSIFICATION.csv",usecols=["reviewText","isPositive"]).head(300)

In [17]:
df.head()

,reviewText,isPositive
0,"PURCHASED FOR YOUNGSTER WHO\nINHERITED MY ""TOO...",1.0
1,unable to open or use,0.0
2,Waste of money!!! It wouldn't load to my system.,0.0
3,I attempted to install this OS on two differen...,0.0
4,I've spent 14 fruitless hours over the past tw...,0.0


In [18]:
df["isPositive"]=df["isPositive"].apply(lambda x:int(x))

In [19]:
df.head()

,reviewText,isPositive
0,"PURCHASED FOR YOUNGSTER WHO\nINHERITED MY ""TOO...",1
1,unable to open or use,0
2,Waste of money!!! It wouldn't load to my system.,0
3,I attempted to install this OS on two differen...,0
4,I've spent 14 fruitless hours over the past tw...,0


In [20]:
df.shape

(300, 2)

In [26]:
import spacy
spacy.cli.download("en_core_web_lg")
nlp = spacy.load("en_core_web_lg")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [29]:
def preprocess_vector(text):
  doc=nlp(text)
  preprocessed_word_list=[token.lemma_ for token in doc if not token.is_stop and token.pos_ not in ["SPACE","X"]]
  return wv.get_mean_vector(preprocessed_word_list)

In [30]:
df["vector"]=df["reviewText"].apply(preprocess_vector)

In [32]:
df.head()

,reviewText,isPositive,vector
0,"PURCHASED FOR YOUNGSTER WHO\nINHERITED MY ""TOO...",1,"[-0.0054304455, -0.021957392, 0.006828725, 0.0..."
1,unable to open or use,0,"[0.048695978, -0.027733276, 0.021992758, 0.019..."
2,Waste of money!!! It wouldn't load to my system.,0,"[0.024200149, 0.039841145, 0.046796847, 0.0457..."
3,I attempted to install this OS on two differen...,0,"[0.0038041144, -0.0014684347, -0.0223611, 0.02..."
4,I've spent 14 fruitless hours over the past tw...,0,"[0.008472721, 0.0012935952, -0.008846553, 0.04..."


In [35]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report

In [38]:
X_train,X_test,y_train,y_test=train_test_split(df["vector"],df["isPositive"],test_size=0.2,random_state=42)
X_train_2d=np.stack(X_train)
X_test_2d=np.stack(X_test)

In [39]:
MultinomialNB_model=Pipeline([
    ("scaler",MinMaxScaler()),
    ("model",MultinomialNB())
    ])
KNeighborsClassifierB_model=Pipeline([
    ("scaler",MinMaxScaler()),
    ("model",KNeighborsClassifier())
    ])

In [40]:
MultinomialNB_model.fit(X_train_2d,y_train)

Pipeline(steps=[('scaler', MinMaxScaler()), ('model', MultinomialNB())])

In [41]:
KNeighborsClassifierB_model.fit(X_train_2d,y_train)

Pipeline(steps=[('scaler', MinMaxScaler()), ('model', KNeighborsClassifier())])

In [42]:
MultinomialNB_model_preds=MultinomialNB_model.predict(X_test_2d)
KNeighborsClassifierB_model_preds=KNeighborsClassifierB_model.predict(X_test_2d)

In [43]:
print(classification_report(y_test,MultinomialNB_model_preds))

              precision    recall  f1-score   support

           0       0.62      0.23      0.33        22
           1       0.67      0.92      0.78        38

    accuracy                           0.67        60
   macro avg       0.65      0.57      0.56        60
weighted avg       0.66      0.67      0.61        60



In [44]:
print(classification_report(y_test,KNeighborsClassifierB_model_preds))


              precision    recall  f1-score   support

           0       0.56      0.68      0.61        22
           1       0.79      0.68      0.73        38

    accuracy                           0.68        60
   macro avg       0.67      0.68      0.67        60
weighted avg       0.70      0.68      0.69        60

